## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-10-08-55-28 +0000,bbc,Ukraine 'ready for elections' if partners guar...,https://www.bbc.com/news/articles/cn5l1474yv7o...
1,2025-12-10-08-54-34 +0000,nyt,Coupang C.E.O. Resigns in Fallout Over Massive...,https://www.nytimes.com/2025/12/10/business/co...
2,2025-12-10-08-52-00 +0000,wsj,Gold Slips Ahead of Powell’s Remarks,https://www.wsj.com/finance/commodities-future...
3,2025-12-10-08-46-00 +0000,wsj,Coupang CEO Resigns Over Data Breach in South ...,https://www.wsj.com/business/retail/coupang-ce...
4,2025-12-10-08-42-00 +0000,wsj,Oil Little Changed as Traders Monitor Ukraine ...,https://www.wsj.com/finance/commodities-future...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2363/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
89,trump,38
0,ukraine,12
248,out,12
317,new,12
134,up,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
82,2025-12-10-01-42-00 +0000,wsj,Eileen Higgins became the first Democrat to wi...,https://www.wsj.com/politics/elections/democra...,91
314,2025-12-09-11-56-07 +0000,nypost,"Trump warns Ukraine is ‘losing’ Russia war, ca...",https://nypost.com/2025/12/09/us-news/trump-sa...,87
99,2025-12-10-00-58-36 +0000,nyt,Trump Calls Europe ‘Decaying’ and Suggests ‘Si...,https://www.nytimes.com/2025/12/09/us/politics...,81
74,2025-12-10-02-08-00 +0000,wsj,President Trump touted the economy as he embar...,https://www.wsj.com/politics/policy/trump-tout...,79
105,2025-12-10-00-37-01 +0000,nypost,Trump kicks off affordability messaging tour a...,https://nypost.com/2025/12/09/us-news/trump-ki...,77


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
82,91,2025-12-10-01-42-00 +0000,wsj,Eileen Higgins became the first Democrat to wi...,https://www.wsj.com/politics/elections/democra...
291,76,2025-12-09-13-30-50 +0000,bbc,Ukraine prepares new peace plan as Zelensky ru...,https://www.bbc.com/news/articles/cg7vdd115vjo...
90,55,2025-12-10-01-14-00 +0000,wsj,Kevin Hassett says he would rely on his own ju...,https://www.wsj.com/economy/central-banking/ke...
206,46,2025-12-09-19-12-00 +0000,wsj,"Bruce Blakeman, a Long Island Republican who t...",https://www.wsj.com/politics/elections/long-is...
148,45,2025-12-09-22-33-02 +0000,nypost,"Netflix, Paramount shares sink as Wall Street ...",https://nypost.com/2025/12/09/media/netflix-pa...
320,41,2025-12-09-11-00-43 +0000,wapo,Have you seen this (AI-generated) man? Police ...,https://www.washingtonpost.com/nation/2025/12/...
300,41,2025-12-09-13-00-00 +0000,wsj,The Bank of Canada is expected to hold steady ...,https://www.wsj.com/articles/bank-of-canada-ex...
32,41,2025-12-10-05-41-55 +0000,nypost,Accused Georgia high school shooter Colt Gray ...,https://nypost.com/2025/12/10/us-news/georgia-...
243,37,2025-12-09-17-02-01 +0000,bbc,Ex-Reform council leader quits following socia...,https://www.bbc.com/news/articles/c773dym3d7do...
303,33,2025-12-09-12-41-57 +0000,bbc,Congress ups pressure to release boat strike v...,https://www.bbc.com/news/articles/c773de38p2go...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
